# **Import all required libraries.**

In [27]:
import numpy as np
from pandas import read_excel
import random
from sklearn.model_selection import train_test_split
from scipy.fftpack import fft
from keras import Input
from keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, Dropout, BatchNormalization, Activation
from tensorflow.keras import optimizers
from keras.models import Model, clone_model, load_model
from keras.utils import plot_model
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, roc_curve, roc_auc_score, ConfusionMatrixDisplay

# **Read data from excel.**

In [6]:
# numeric representation function
def EIIP(data_, n):
    row=len(data_)
    data= np.zeros([row,n])
    for i in range(row):
        for j in range(n):
            char= data_[i][0][j]
            if char == 'A':
                data[i, j]= 0.1260
            if char == 'C':
                data[i, j]= 0.1340
            if char == 'G':
                data[i, j]= 0.0806
            if char == 'T':
                data[i, j]= 0.1335
    return data

# chatching data from excel file
seq= np.array( read_excel('/content/filtered_data.xlsx', header= None) )
# set the data shape
m, n= 1156, 26592
all_data= EIIP(seq, n)
# create the labels: 1= Covid-19, 0= Other Human Coronavirus
target= np.array([np.full(int(m/2), 1), np.full(int(m/2), 0)])
target= target.reshape(m,1)

# **Bring testing data out of the data**

In [10]:
# seed for reproducibility
seed= 5
random.seed(seed)
# acquire testing data/labels from dataset
x= random.sample(range(1156), 156)
x= np.sort(x, axis=0)
testing= all_data[(x)]
testing_label= target[x]
# remove obtained testing data from training/validation data
data= list(all_data)
data_label= list(target)
for i in range(len(x)):
  data.pop( x[ -(i+1) ] )
  data_label.pop( x[ -(i+1) ] )

data= np.array(data)

# **Enlarge the data by "data slicing"**

In [ ]:
# data slicing function
def sliced_array(data, data_label, K, N): # k= window size, N= number of features
    new_data = np.zeros( (data.shape[0]*(int(N/K)), 256) )
    new_target= [0]*( data.shape[0]*(int(N/K)) )
    temp = []
    p= 0
    for i in range(data.shape[0]):
      temp.clear()
      for j in range(data.shape[1]):
          temp.append( data[i,j] )
          if ((j+1) % K) == 0:
              new_data[p]= temp
              new_target[p]= data_label[i][0]
              p= p+1
              temp.clear()
    return new_data, new_target

# call the function
data, data_label = sliced_array(data, data_label, 256, n)
testing, testing_label = sliced_array(testing, testing_label, 256, n)
# check about new shapes
print('shape of data=%s, labels size=%d' %(data.shape, len(data_label)) )
print('shape of testing data=%s, testing labels size=%d' %(testing.shape, len(testing_label)) )

# **Split data into teaining and validation data**

In [12]:
# select percentage of splitting (0.3 used)
x_train, val_train, y_train, y_test= train_test_split(data, data_label,
                                                      test_size=0.3, random_state=10,
                                                      shuffle=True, stratify=data_label)
#convert y_train from list to array
y_train= np.array(y_train)
y_test= np.array(y_test)
testing_label= np.array(testing_label)

# **Apply FFT method with removing DC component**

In [23]:
for i in [x_train, val_train, testing]:
    # find the sample mean
    x_mean= np.mean(i, axis=1)
    # apply broadcasting
    x_mean_pro= np.tile(x_mean, (256,1))
    # sabtract sample mean from org sample to remove bias
    x_DC_removed= i - x_mean_pro.T
    # create FFT data
    if   len(i) == len(x_train):
        x_train_FFT= np.abs( fft(x_DC_removed, axis=1) )
    elif len(i) == len(val_train):
        val_train_FFT= np.abs( fft(x_DC_removed, axis=1))
    elif len(i) == len(testing):
        testing_FFT= np.abs( fft(x_DC_removed, axis=1))

# **Build the model architecture**

In [ ]:
# layers per block function
def creation(layer_in, filters, kernel_size, padding):
    layer_in= Conv1D(filters=filters, kernel_size=kernel_size, strides=1, padding=padding)(layer_in)
    layer_in= BatchNormalization()(layer_in)
    layer_in= Activation('relu')(layer_in)
    layer_in= MaxPooling1D(2, strides=2)(layer_in)
    layer_in= Dropout(0.3)(layer_in)
    return layer_in
# build blocks of the model
inputs= Input(shape=( x_train.shape[1], 1))
block_1= creation(inputs, filters=64, kernel_size=10, padding='valid')
block_2= creation(block_1, 128, 16, 'same')
block_3= creation(block_2, 128, 16, 'same')
block_4= creation(block_3, 128, 16, 'same')
block_5= creation(block_4, 64,  16, 'same')
block_6= creation(block_5, 32,  16, 'same')
block_7= creation(block_6, 32,  16, 'same')
fC_layers_8= Flatten()(block_7)
fC_layers_9= Dense(320)(fC_layers_8)
fC_layers_10= Dropout(0.3)(fC_layers_9)
fC_layers_11= Dense(1, activation='sigmoid')(fC_layers_10)
# construct the model
model= Model(inputs, fC_layers_11)
learning_rate= 0.003
optimizer= optimizers.Adam(learning_rate= learning_rate)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
# visualize the model architecture
model.summary()
plot_model(model, show_shapes=True)

# **Install scikeras library to create estimator**

In [ ]:
!pip install scikeras

# **Apply grid search for hyperparameters tuning**

In [ ]:
# call the necessary libraries
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
# set the numeric data hyperparameters to select
batch_size= [256, 512, 1024,2048, 4096]   # [256, 512, 1024] for FFT data
epochs= [15, 20, 30, 50, 75, 100, 200, 250]  # [35, 45, 55] for FFT data
learning_rate= [0.0005, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3] # for both
param_grid= dict(batch_size=batch_size, epochs=epochs, learning_rate=learning_rate)
estimator= KerasClassifier(model=model, learning_rate=learning_rate, batch_size=batch_size, epochs=epochs, verbose=0)
optimal= GridSearchCV(estimator, param_grid=param_grid, n_jobs=None, verbose=2, cv=3)

# Fit the model to find the optimal hyperparameters

In [ ]:
optimal_his= optimal.fit(X= x_train, y=y_train) # for FFT data, X= x_train_FFT

# **Evaluate the hyperparameters scores**

In [ ]:
mean= optimal_his.cv_results_['mean_test_score']
time= optimal_his.cv_results_['mean_score_time']
std= optimal_his.cv_results_['std_test_score']
params= optimal_his.cv_results_['params']
for a,b,c,d in zip(mean, time, std, params):
  print('mean(%f), time(%f), std(%f), parameters(%s)'%(a,b,c,d))
print('best score is %f with parameters %s'%(optimal_his.best_score_, optimal_his.best_params_))

# **Finally, fit the model with selected hyperparameters**


In [ ]:
# copy the model to fit it with two different data
M= clone_model(model)
optimizer= optimizers.Adam(learning_rate= 0.003)
M.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
# fit the numeric data
his= M.fit(x_train, y_train, batch_size=256, epochs=200, verbose=1,
         validation_data=[val_train, y_test], workers=0) # for FFT data: batch_size=512, epochs=55

# **Plot loss-accuracy curve**

In [ ]:
tr_acc = his.history['accuracy']
tr_loss = his.history['loss']
val_acc = his.history['val_accuracy']
val_loss = his.history['val_loss']
Epochs = [i+1 for i in range( len(tr_acc) )]
# Plot training history
plt.figure(figsize= (20, 8))
plt.style.use('fivethirtyeight')

plt.subplot(1, 2, 1)
plt.plot(Epochs, tr_loss, 'r', label= 'Training loss')
plt.plot(Epochs, val_loss, 'g', label= 'Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(Epochs, tr_acc, 'r', label= 'Training Accuracy')
plt.plot(Epochs, val_acc, 'g', label= 'Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.tight_layout
plt.show()

# **Score the model on validation data**

In [ ]:
score= M.evaluate(val_train, y_test, verbose=1)
print('score of numeric data:\n loss=%f, accuracy=%f' %(score[0], score[1]) )

# **Predict testing data**

In [ ]:
prediction= M.predict(testing, batch_size=256 ) # for FFT data: testing_FFT, batch_size=512
# convert the results from float to int
prediction= np.int_(prediction)

# **Show the classification report and confusion matrix**


In [ ]:
clf= classification_report(testing_label, prediction)
cm= confusion_matrix(testing_label, prediction)
[tp, fp], [fn, tn]= confusion_matrix(testing_label, prediction)
print(cm)
print('tp=%d\nfp=%d\nfn=%d\ntn=%d' %(tp, fp, fn, tn))
print(clf)

# **Display the confusion matrix**

In [ ]:
disp= ConfusionMatrixDisplay(cm, display_labels=['Covid19','Other Coronavirus'])
disp.plot(cmap='gray')

# **Determine all metrics**

In [ ]:
print('accuracy=', ( accuracy_score(testing_label, prediction)*100) )
print('specificity=', ( tn/(tn+fp) )*100  )
sensitivity= ( tp/(tp+fn) )*100
print('sensitivity=', sensitivity)
precision= precision_score(testing_label, prediction) *100
print('precision=', precision)
F_score= (2*precision*sensitivity) / (precision+sensitivity)
print('F-score=', F_score)

# **Plot ROC curve and AUC score**

In [ ]:
fpr, tpr, threshold= roc_curve(testing_label, prediction)
auc= roc_auc_score(testing_label, prediction)
plt.style.use('default')
plt.plot(fpr,tpr, label= 'FFT & Numeric AUC='+str(auc))
plt.legend(loc=4)
plt.xlim([-0.05,1])
plt.ylim([0,1.04])
plt.grid()
plt.xlabel('1-specificity( fpr)')
plt.ylabel('sensitivity (tpr)')
plt.title('ROC')